# Mapping Done

In [1]:
#import spark as sp
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.functions import col, coalesce

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [2]:
spark = SparkSession.builder.appName("cdw_sapp_branch.json").getOrCreate()
df_branch = spark.read.format("json") \
    .option("header", True) \
    .load("cdw_sapp_branch.json")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 11:17:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/28 11:17:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/28 11:17:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df_branch.show(5)
df_branch.printSchema()

+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|    Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL| Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|      14th Street|     19406|2018-04-18T16:51:...|
+-----------------+-----------+-

Mapping Logics

In [4]:
df_branch.show(2)

+-----------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|  Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|    Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
+-----------+-----------+------------+------------+------------+-----------------+----------+--------------------+
only showing top 2 rows



In [5]:
#Change the format of phone number to (XXX)XXX-XXXX
df_branch = df_branch.withColumn('BRANCH_PHONE', concat(lit('('), df_branch['BRANCH_PHONE'].substr(1,3), lit(')'),df_branch['BRANCH_PHONE'].substr(4,3), lit('-'),df_branch['BRANCH_PHONE'].substr(7,4)))

df_branch.show(2)

+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|    Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
+-----------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
only showing top 2 rows



In [6]:
# Replace null values in "branch_zip" column with "99999"
df_branch = df_branch.withColumn("branch_zip", when(df_branch["branch_zip"].isNull(), "99999").otherwise(df_branch["branch_zip"]))

# Show the updated DataFrame
df_branch.show()



+-----------------+-----------+------------+-------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|branch_zip|        LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|(123)466-3064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|(123)484-9701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|       

In [7]:
df_branch.write.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "creditcard_capstone.branch") \
          .option("user", "root") \
            .option("password", "password") \
            .mode("overwrite")\
            .save()
df_branch.show(1)

+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|branch_zip|        LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN| Bridle Court|     55044|2018-04-18T16:51:...|
+-----------+-----------+------------+-------------+------------+-------------+----------+--------------------+
only showing top 1 row

